In [ ]:
"""
Custom notebook to:
    - plot the combined density PDFs of several cluster simulations, comparing different driving,
      at the same RMS Mach number.

Requires:
    - 'cluster_?????_rho.pdf' files. The density PDF at each timestep, for each simulation. (Can
       be generated using phantomanalysis.)
    - 'rmsmach_vs_time.data' files for each simulation. (Use 'rmsmach_vs_time.sh' to generate it.)
Directories for each can be specified in the first cell, as a list of strings.

Written by:
David Liptai, Monash University.
2015-2016
"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp as kstest
import os
import subprocess
from scipy.interpolate import interp1d
%matplotlib

#Plotting stuff nicely
from matplotlib import gridspec
from IPython.display import display, Math, Latex
import math
from math import sqrt, cos, sin, pi
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
from matplotlib.ticker import FormatStrFormatter, LinearLocator, NullFormatter, NullLocator, MultipleLocator
import matplotlib.ticker
import matplotlib.colors
from matplotlib.font_manager import FontProperties
from matplotlib import rc, text
plt.close('all')
fig_width_pt = 504   #245.27        # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inches
golden_mean = (np.sqrt(5)-1.0)/2.0      # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height =fig_width*golden_mean       # height in inches
fig_size = [fig_width,fig_height]

fontsize=18
#fig_size = [18,16]
params = {'backend': 'pdf',
          'axes.labelsize': fontsize,
          'lines.markersize': 4,
          'font.size': fontsize,
          'xtick.major.size':8,
          'xtick.minor.size':4,
          'ytick.major.size':8,
          'ytick.minor.size':4,
          'xtick.major.width':2,
          'ytick.major.width':2,
          'xtick.minor.width':1,
          'ytick.minor.width':1,
          'lines.markeredgewidth':1,
          'axes.linewidth':1.2,
          'legend.fontsize': fontsize-3,
          'xtick.labelsize': fontsize-2,
          'ytick.labelsize': fontsize-2,
          'savefig.dpi':200,
          'path.simplify':True,
          'font.family': 'serif',
          'font.serif':'Times',
          'text.latex.preamble': [r'\usepackage{amsmath}'],
          'text.usetex':True,
          'axes.color_cycle': ['b', 'lime', 'r', 'purple', 'g', 'c', 'm', 'orange', 'darkblue', \
                               'darkcyan', 'y','orangered','chartreuse','brown','deeppink','lightgreen', 'k'],
          #'font.serif':cm,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
plt.clf()
gs = gridspec.GridSpec(1,1)
plt.close('all')

def BASH(command):
	return subprocess.check_output(command,shell=True).decode().strip()

utime       =  1.487E+13/60./60./24./365. 				# in years
t_ff        =  0.806588045/2.*utime						# in years


solenoidals = ['sol' +str(i) for i in range(1,8)]
compressives= ['comp'+str(i) for i in range(1,8)]
dir_names = solenoidals+compressives
dir_home = BASH('echo $HOME')
dir_prefix = '/Volumes/dlip1/runs/' #dir_home+'/dlip1/runs/'
#datafile='rmsmach_vs_time.data'
#datafile='sigma_vs_time.data'

def load_data_file(datafile,folders):
    # Input: datafile, list of folder names e.g. [sol1,sol2,....]
    # Output: data array
    L_shortest=1e10
    L_longest=0
    data=np.zeros([1e6,len(folders)+1])
    for i in range(0,len(folders)):
        name = folders[i]
        temp = np.genfromtxt(dir_prefix+name+'/'+datafile,skip_footer=1)
        L_shortest = min(len(temp),L_shortest)
        if len(temp)>L_longest:
            data[:len(temp),0] = temp[:len(temp),0] #Rewrite time column
        data[:len(temp),i+1] = temp[:len(temp),1]
        L_longest  = max(len(temp),L_longest)
    shortest_length=L_shortest
    data=data[:L_longest,:]
    return data, shortest_length

#sol_sig,lsol_sig = load_data_file(datafile='sigma_vs_time.data',folders=solenoidals)
sol_rms,lsol_rms = load_data_file(datafile='rmsmach_vs_time.data',folders=solenoidals)
#comp_sig,lcomp_sig = load_data_file(datafile='sigma_vs_time.data',folders=compressives)
comp_rms,lcomp_rms = load_data_file(datafile='rmsmach_vs_time.data',folders=compressives)

In [ ]:
t1=sol_rms[:lsol_rms,0]
y1=sol_rms[:lsol_rms,1:]

# Choose the RMS Mach number you want to compare at...
MACHNUMBER=5.5

L_shortest=1e10
L_longest=0
solPDF=np.zeros([1e6,len(solenoidals)+1])
for i in range(0,len(solenoidals)):
    name = solenoidals[i]
    nmach5p5=np.argmax(sol_rms[:lsol_rms,i+1]<MACHNUMBER)
    nmach5p5=str(nmach5p5).rjust(5,'0')
    temp = np.genfromtxt(dir_prefix+name+'/'+'cluster_'+nmach5p5+'_rho.pdf',skip_header=1)
    L_shortest = min(len(temp),L_shortest)
    if len(temp)>L_longest:
        solPDF[:len(temp),0] = temp[:len(temp),0]
    solPDF[:len(temp),i+1] = temp[:len(temp),1]
    L_longest  = max(len(temp),L_longest)
shortest_length=L_shortest
solPDF=solPDF[:L_longest,:]


x1 = np.log10(solPDF[:,0]*6.775E-20)
y1 = solPDF[:,1:]
y1_av = np.mean(y1,axis=1)
y1_std= np.std(y1,axis=1)

L_shortest=1e10
L_longest=0
compPDF=np.zeros([1e6,len(compressives)+1])
for i in range(0,len(compressives)):
    name = compressives[i]
    nmach5p5=np.argmax(comp_rms[:lcomp_rms,i+1]<MACHNUMBER)
    nmach5p5=str(nmach5p5).rjust(5,'0')
    temp = np.genfromtxt(dir_prefix+name+'/'+'cluster_'+nmach5p5+'_rho.pdf',skip_header=1)
    L_shortest = min(len(temp),L_shortest)
    if len(temp)>L_longest:
        compPDF[:len(temp),0] = temp[:len(temp),0]
    compPDF[:len(temp),i+1] = temp[:len(temp),1]
    L_longest  = max(len(temp),L_longest)
shortest_length=L_shortest
compPDF=compPDF[:L_longest,:]

x2 = np.log10(compPDF[:,0]*6.775E-20)
y2 = compPDF[:,1:]
y2_av = np.mean(y2,axis=1)
y2_std= np.std(y2,axis=1)

In [ ]:
# Uncomment to make white on black/transparent
'''
prms = {'text.color':'white',
        'axes.facecolor':'black',
        'axes.edgecolor':'white',
        'axes.labelcolor':'white',
        'xtick.color':'white',
        'ytick.color':'white',
        'grid.color':'white',
        'savefig.transparent' : True
       }
plt.rcParams.update(prms)
'''

plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)#, axisbg='black')
#ax.patch.set_facecolor('black')

#ax.spines['top'].set_color('white')
#ax.xaxis.label.set_color('white')
#ax.tick_params(axis='x', colors='white')

plt.xlim(xmax=max(max(x1),max(x2)),xmin=min(min(x1),min(x2)))
plt.xlim(xmin=-22,xmax=-15)
plt.ylim(ymin=-0.0001,ymax=0.4)
plt.plot(x1,y1_av,'g',label='Solenoidal')
plt.fill_between(x1, y1_av+y1_std, y1_av-y1_std,color='g',alpha=0.1)
plt.plot(x2,y2_av,'r',label='Compressive')
plt.fill_between(x2, y2_av+y2_std, y2_av-y2_std,color='r',alpha=0.1)
plt.ylabel('PDF')
plt.xlabel(r'$\log \rho $ ~~[log(g cm^{-3})]')
plt.legend(loc='upper right',frameon=False)
plt.tick_params(which='both',axis='both',pad=8)
plt.tight_layout()
plt.show()
#plt.savefig('/Users/david/Dropbox/project_honours/pdfsmach5p5.pdf')

In [ ]:
rcParams.keys()